In [216]:
from bs4 import BeautifulSoup
import urllib
import itertools
import re
import json

In [253]:
website = urllib.urlopen('http://meetings.aps.org/Meeting/DAMOP16/APS_epitome')
soup = BeautifulSoup(website, 'html.parser')

In [269]:
session_list = [tag.find('b').text.strip() for tag in soup.find_all("p", { "class" : "epi" })]
print session_list
#     break

[u'1A', u'1B', u'1C', u'A1', u'B2', u'B3', u'B4', u'B5', u'B6', u'B7', u'B8', u'B9', u'C2', u'C3', u'C4', u'C5', u'C6', u'C7', u'C8', u'C9', u'D1', u'D2', u'E5', u'E7', u'F1', u'G2', u'G3', u'G4', u'G5', u'G6', u'G7', u'G8', u'G9', u'H2', u'H3', u'H4', u'H5', u'H6', u'H7', u'H8', u'H9', u'J2', u'J3', u'J4', u'J5', u'J6', u'J7', u'J8', u'J9', u'K1', u'L2', u'M2', u'M3', u'M4', u'M5', u'M6', u'M7', u'M8', u'M9', u'N2', u'N3', u'N4', u'N5', u'N6', u'N7', u'N8', u'N9', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'Q1', u'P11', u'S1', u'T2', u'T3', u'T5', u'T6', u'T7', u'T8', u'T9', u'U2', u'U3', u'U5', u'U6', u'U9', u'1A', u'1B', u'1C', u'A1', u'B2', u'B3', u'B4', u'B5', u'B6', u'B7', u'B8', u'B9', u'C2', u'C3', u'C4', u'C5', u'C6', u'C7', u'C8', u'C9', u'D1', u'D2', u'E5', u'E7', u'F1', u'G2', u'G3', u'G4', u'G5', u'G6', u'G7', u'G8', u'G9', u'H2', u'H3', u'H4', u'H5', u'H6', u'H7', u'H8', u'H9', u'J2', u'J3', u'J4', u'J5', u'J6', u'J7', u'J8', u'J9', u'K1', u'L2', u'M

In [5]:
def get_address(session):
    return 'http://meetings.aps.org/Meeting/DAMOP16/Session/{}?showAbstract'.format(session)
print get_address('H6')

http://meetings.aps.org/Meeting/DAMOP16/Session/H6?showAbstract


In [241]:
def download_session(session_number):
    website = urllib.urlopen(get_address(session_number))
    soup = BeautifulSoup(website, 'html.parser')

    # First table has the session name
    # Have to use re twice because the first time it returns the "Session..." part
    myregexp = re.compile("Session {}: (.*)".format(session_number))
    session_name = myregexp.match(soup.find(text=myregexp)).group(1)
    # print session_name

    # Second table has the abstracts.
    abstracts = []

    for x in soup.find_all(href=re.compile('/Meeting/DAMOP16/Session/{}.'.format(session_number))):
        results = re.search('{}.([0-9])*: (.*)'.format(session_number), x.text)
        current_number = results.group(1)
        current_title = results.group(2)
        current_abstract = x.find_next().find('p').text.replace('[Preview Abstract]', '').strip()
    #     print (current_number, current_title, current_abstract)
        abstracts.append({'number': current_number,
                          'title': current_title,
                          'abstract': current_abstract})
    return {'name': session_name,
           'number': session_number,
           'abstracts': abstracts,
          }

In [225]:
website = urllib.urlopen(get_address('A1'))
soup = BeautifulSoup(website, 'html.parser')

In [239]:
for x in soup.find_all(href=re.compile('/Meeting/DAMOP16/Session/{}.'.format('A1'))):
    print x.find_next().find('p').text.replace('[Preview Abstract]', '').strip()
#     break

Opening remarks for the the DAMOP16 Conference.
Ultracold atomic gases confined to optical lattices have proven to be highly versatile and tunable systems for realizing novel quantum states of matter. We are using Fermi gases of $^6$Li atoms in our laboratory to explore several goals related to the strong correlations that arise in these systems. We have realized the Hubbard model, which has long been suspected of containing the essential ingredients of high temperature superconductivity. We measured the compressibility of the Mott insulating phase that occurs near half filling (1 atom/site), thus demonstrating the excitation gap of the Mott insulator. Progress in this field, however, has been hampered by an inability to cool to low enough temperatures to achieve the most ambitious goals. To address this problem, we have developed the compensated optical lattice method to enable evaporative cooling in the lattice. With this method, we have cooled the Mott insulator sufficiently far to 

In [251]:
data = []
for session in session_list:
    print session
    data.append(download_session(session))
#     break

A1
C2
C3
C4
C5
C6
C7
C8
C9
B2
B3
B4
B5
B6
B7
B8
B9


In [267]:
print download_session('D1')

{'abstracts': [{'abstract': u'', 'number': u'1', 'title': u'SPECTROSCOPY, LIFETIMES, OSCILLATOR STRENGTHS'}, {'abstract': u'We present high precision spectroscopic results on the 2s2p J$=$1 singlet \nstate in neutral beryllium-9. Combined with theoretical predictions this \nmeasurement serves as a test of quantum electrodynamics and various \ntheoretical methods for predicting the energy of this state. Our \nexperimental setup consists of an oven at 1200C that produces a beam of \nberyllium atoms. The singlet state is probed transverse to the atomic beam \nwith 235nm light from a frequency quadrupled titanium sapphire laser, where \nthe frequency doubled light at 470nm is stabilized to an ultra low expansion \ncavity. We also present our progress on spectroscopy on the lowest triplet \nstates and the ionization threshold.', 'number': u'2', 'title': u'Testing quantum electrodynamics in the lowest singlet state of neutral beryllium-9'}, {'abstract': u'One challenge in engineering long-li

In [252]:
with open("damop.json", "w") as f:
    json.dump(data, f)